## some header

In [ ]:
## When the model is ready, it will be pushed here and evaluated on MoleculeACE tasks

### setup

In [ ]:
# installs
#!pip install MoleculeACE torch-geometric transformers

* CHEMBL234 - Dopamine D3 receptor
* CHEMBL4203 - Dual specificity protein kinase
* CHEMBL2047 - Farnesoid X receptor
* CHEMBL4616 - Ghrelin receptor
* CHEMBL264 - Histamine H3 receptor
* CHEMBL2835 - Janus kinase 1
* CHEMBL4792 - Orexin receptor 2

In [ ]:
from MoleculeACE import MLP, Data, Descriptors, calc_rmse, calc_cliff_rmse, get_benchmark_config

import datamol as dm
from molfeat.calc import FP_FUNCS, FPCalculator
from molfeat.trans.concat import FeatConcat
from molfeat.trans import MoleculeTransformer

In [ ]:
datasets = 'CHEMBL234_Ki', 'CHEMBL4203_Ki', 'CHEMBL2047_EC50', 'CHEMBL4616_EC50', 'CHEMBL264_Ki', 'CHEMBL2835_Ki', 'CHEMBL4792_Ki'
algorithm = MLP
dataset = 'CHEMBL4203_Ki'
data = Data(dataset)
descriptor = Descriptors.ECFP
# Load data

# Get the already optimized hyperparameters
hyperparameters = get_benchmark_config(dataset, algorithm, descriptor)

In [ ]:
train_smiles = data.smiles_train
test_smiles = data.smiles_test
featurizer = MoleculeTransformer(FPCalculator('ecfp', length=2048, radius=4))
featurized_train = torch.as_tensor(featurizer(train_smiles), dtype = torch.float32)
featurized_test = torch.as_tensor(featurizer(test_smiles), dtype=torch.float32)

In [ ]:
# Train and a model
model = algorithm(**hyperparameters)

model.train(data.x_train, data.y_train)
y_hat = model.predict(data.x_test)

# Evaluate your model on activity cliff compounds
rmse = calc_rmse(data.y_test, y_hat)
rmse_cliff = calc_cliff_rmse(y_test_pred=y_hat, y_test=data.y_test, cliff_mols_test=data.cliff_mols_test)

print(f"rmse: {rmse}")
print(f"rmse_cliff: {rmse_cliff}")